### Project 4: Continued 
In the previous notebook, I did all the processing I could within the memory limitations of my laptop. I exported the cleaned CSV and starting with a clean kernel to One Hot encode the data so that the customers are the index and each column is a movie. I will also explore PCA. 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#read the previously cleaned csv
df = pd.read_csv('combined_data_cleaned.csv')
df = df.astype('uint32')

### Continuing to Reduce Data Size 
Need to reduce size of data still. In the previous notebook, somewhat methodical methods of compression were implemented. The data was reduced a considerable amount. But it is still too large for some of the operations I want to complete. At this point we'll randomly drop customers and movies using the function below.

In [20]:
import typing as T
import random as rand

#Drop random indices from given list
def drop_random(values: list, length: int, proportion: float):
    num_to_drop = length -  int(length*proportion)
    to_drop = rand.sample(range(length), num_to_drop)
    return np.delete(values, to_drop, axis=None)
    

In [11]:
%%time
#get customer and movie unqique values
customers = df.Cust_Id.value_counts().index
movies = df.Movie_Id.value_counts().index

# get random customers and movies to drop
customers_to_drop = drop_random(customers, len(customers), .1)
movies_to_drop = drop_random(movies, len(movies), .3)

#drop the random customers and movies
df_trimmed = df[df['Cust_Id'].isin(customers_to_drop)]
df_trimmed = df_trimmed[df_trimmed['Movie_Id'].isin(movies_to_drop)]
del df

CPU times: user 1.83 s, sys: 92 ms, total: 1.93 s
Wall time: 1.91 s


In [12]:
print("Customer count before drop:\t", len(customers))
print("Movie count before drop:\t", len(movies))
print("Customer count after drop:\t", len(df_trimmed.Cust_Id.value_counts().index))
print("Movie count after drop:\t\t", len(df_trimmed.Movie_Id.value_counts().index))

Customer count before drop:	 291590
Movie count before drop:	 16031
Customer count after drop:	 29159
Movie count after drop:		 4793


In [13]:
df_trimmed.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1038392 entries, 1025 to 35021619
Columns: 3 entries, Cust_Id to Movie_Id
dtypes: uint32(3)
memory usage: 19.8 MB


In [14]:
#rearrange data frame so that customers and rows and movies are columns
df_pivot = df_trimmed.pivot_table(index = 'Cust_Id', columns = 'Movie_Id', values = 'Rating')
del df_trimmed

In [15]:
df_pivot.shape

(29159, 4793)

In [16]:
df_pivot

Movie_Id,4,5,10,12,16,18,21,23,24,31,...,17741,17746,17751,17752,17760,17764,17765,17766,17769,17770
Cust_Id,,,,,,,,,,,,,,,,,,,,,
192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_pivot = df_pivot.fillna(0)

In [19]:
# export reformatted data 
df_pivot.to_csv('data_pivot.csv')